In [1]:
import docker
d = docker.DockerClient()


[b'yarn run v1.22.19',
 b'$ npm-run-all check-db update-tracker start-server',
 b'$ node scripts/check-db.js',
 b'\xe2\x9c\x93 DATABASE_URL is defined.',
 b'\xe2\x9c\x97 Unable to connect to the database.',
 b'error Command failed with exit code 1.',
 b'info Visit https://yarnpkg.com/en/docs/cli/run for documentation about this command.',
 b'ERROR: "check-db" exited with 1.',
 b'error Command failed with exit code 1.',
 b'info Visit https://yarnpkg.com/en/docs/cli/run for documentation about this command.',
 b'yarn run v1.22.19',
 b'$ npm-run-all check-db update-tracker start-server',
 b'$ node scripts/check-db.js',
 b'\xe2\x9c\x93 DATABASE_URL is defined.',
 b'yarn run v1.22.19',
 b'$ npm-run-all check-db update-tracker start-server',
 b'$ node scripts/check-db.js',
 b'\xe2\x9c\x93 DATABASE_URL is defined.',
 b'\xe2\x9c\x93 Database connection successful.',
 b'\xe2\x9c\x93 Database tables found.',
 b'Prisma schema loaded from prisma/schema.prisma',
 b'Datasource "db": PostgreSQL datab

In [2]:
import socketio

In [3]:
import asyncio
import socketio

sio = socketio.Client()




@sio.on('*')
def catch_all(event, data):
    print(data)



sio.connect('http://localhost:5000/')




ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Read timed out. (read timeout=5)

In [ ]:
sio.emit("subscribe_stats", "backend")

packet queue is empty, aborting


In [ ]:
sio.wait()